In [1]:
import utils
import data_utils
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from pygam import LogisticGAM
import statsmodels.api as sm

%matplotlib notebook

# Automating the proess of finding the filename
dataFolder = 'data'
plotsFolder = 'plots'
state = 'TX'
policeDept = 'statewide' # 'statewide' means state patrol

# Choose column names to load
colNamesList = ['date',
                'time',
#                 'location',
#                 'lat', # about 40% are nan
#                 'lng',
                'county_name', # just use categorical metro, micro, other for all location vars
#                 'district',
#                 'precinct',
#                 'region',
                'subject_race',
                'subject_sex',
#                 'officer_id_hash',
#                 'type',
                'violation',
                'citation_issued',
                'warning_issued', 
                'contraband_found',
                'contraband_drugs',
                'contraband_weapons',
                'search_conducted',
                'search_vehicle']

In [2]:
# Automating the creation of rowName:dtype dict
# This throws up a lot of stupid warnings/errors right now!
# df = pd.read_csv(csvFilepath, nrows=5, names=colNamesList)
# dtypeDict = df.dtypes.to_dict()
# dtypeDict

dtypeDict = {'raw_row_number': 'int64', 
             'date': 'object',
             'time': 'object',
             'location': 'object',
             'lat': 'float64',
             'lng': 'float64',
             'county_name': 'object',
             'district': 'object',
             'precinct': 'object',
             'region': 'float64',
             'subject_race': 'object',
             'subject_sex': 'object',
             'officer_id_hash': 'object',
             'type': 'object',
             'violation': 'object',
             'citation_issued': 'bool',
             'warning_issued': 'bool',
             'outcome': 'object',
             'contraband_found': 'object',
             'contraband_drugs': 'object',
             'contraband_weapons': 'object',
             'search_conducted': 'object',
             'search_vehicle': 'object',
             'search_basis': 'object',
             'vehicle_color': 'object',
             'vehicle_make': 'object',
             'vehicle_model': 'object',
             'vehicle_type': 'object',
             'vehicle_year': 'float64',
             'raw_HA_RACE_SEX': 'object',
             'raw_HA_SEARCH_PC_boolean': 'object',
             'raw_HA_SEARCH_CONCENT_boolean': 'object',
             'raw_HA_INCIDTO_ARREST_boolean': 'object',
             'raw_HA_VEHICLE_INVENT_boolean': 'object'}


        
# search_vehicle only has False and nan
# type is just vehicular
# violation column is of interest - reason for stop (split by |?)

In [3]:
data = data_utils.TrafficStopData(dataFolder, state, policeDept, colNamesList, dtypeDict)

loading time: 267


In [4]:
df = data.preprocess()

preprocessing time: 375


In [5]:
X_train,y_train,X_test,y_test = data.get_train_test_sets()

In [9]:
X_test = sm.add_constant(X_test)
X_train = sm.add_constant(X_train)

In [10]:
from sklearn.linear_model import LogisticRegressionCV

In [12]:
start = time.time()
clf = LogisticRegressionCV()
clf.fit(X_train, y_train)
end = time.time()
print('training time: %d'%(end-start))

training time: 470


In [13]:
clf.get_params()

{'Cs': 10,
 'class_weight': None,
 'cv': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1.0,
 'l1_ratios': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'refit': True,
 'scoring': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0}

In [14]:
# training accuracy
y_train_pred = clf.predict(X_train)
np.mean(y_train_pred == y_train)

0.8739182400135589

In [15]:
# test accuracy
y_test_pred = clf.predict(X_test)
np.mean(y_test_pred == y_test)

0.8739572736098262

In [17]:
C = clf.C_

In [18]:
start = time.time()
clf = sm.Logit(y_train, X_train).fit_regularized()
end = time.time()
print('training time: %d'%(end-start))

/Users/nicoletan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/nicoletan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1852: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.32838203669073485
            Iterations: 229
            Function evaluations: 235
            Gradient evaluations: 229


LinAlgError: Singular matrix

In [19]:
clf.summary()

AttributeError: 'LogisticRegressionCV' object has no attribute 'summary'

In [ ]:
# training accuracy
y_train_pred = clf.predict(X_train)
np.mean(y_train_pred == y_train)

In [ ]:
# test accuracy
y_test_pred = clf.predict(X_test)
np.mean(y_test_pred == y_test)